In [33]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ta

# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

API_KEY = None
SECRET_KEY = None

import os
from dotenv import load_dotenv

load_dotenv(override=True)

if API_KEY is None:
    API_KEY = os.environ.get('ALP_API_KEY')

if SECRET_KEY is None:
    SECRET_KEY = os.environ.get('ALP_SEC_KEY')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)

from alpaca.data.enums import Adjustment

In [35]:
from regime_tickers import custom_vol_subset, custom_exp_subset

In [36]:
stock_historical_data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# alpaca has no older data than 2016-01-04 for this symbol set
earliest_date = datetime(2016, 1, 4, tzinfo=ZoneInfo('America/New_York'))
# earliest_timestamps_by_symbol = df_adj.reset_index().groupby('symbol')['timestamp'].min()

req = StockBarsRequest(
    symbol_or_symbols = list(set(custom_vol_subset + custom_exp_subset + ['SPY'])),  # add SPY for market reference
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Day), 
    start = earliest_date,                    
    limit = None,    
    adjustment=Adjustment('all') # adjust for splits and dividends                                           
)
df_adj = stock_historical_data_client.get_stock_bars(req).df.reset_index().set_index('timestamp')
df_adj = df_adj.sort_values(by=['symbol', 'timestamp']) # Ensure sorted for correct rolling calcs


In [37]:
pd.set_option('display.max_rows', 300)
print("Total NANs:", df_adj.pivot(columns="symbol").isna().sum().sum())
print(df_adj.pivot(columns="symbol").isna().sum())

pd.reset_option('display.max_rows')

Total NANs: 0
             symbol
open         BND       0
             CPER      0
             GLD       0
             HYG       0
             IWM       0
             SCHP      0
             SPY       0
             VIXY      0
             XLI       0
high         BND       0
             CPER      0
             GLD       0
             HYG       0
             IWM       0
             SCHP      0
             SPY       0
             VIXY      0
             XLI       0
low          BND       0
             CPER      0
             GLD       0
             HYG       0
             IWM       0
             SCHP      0
             SPY       0
             VIXY      0
             XLI       0
close        BND       0
             CPER      0
             GLD       0
             HYG       0
             IWM       0
             SCHP      0
             SPY       0
             VIXY      0
             XLI       0
volume       BND       0
             CPER      0
             GLD

In [38]:
############## windows = [5, 20, 60]
windows = [20]

In [39]:
# Calculate Daily Log Returns
df_adj['log_returns'] = df_adj.groupby('symbol')['close'].transform(lambda x: np.log(x / x.shift(1)))

In [40]:
z_window = 60

In [41]:
# Rolling Standard Deviation (Volatility). For further computations, later we will use z-scores of these values only
for w in windows:
    df_adj[f'rolling_std_20'] = df_adj.groupby('symbol')['log_returns'].transform(
        lambda x: x.rolling(window=w, min_periods=w).std()
    )

    df_adj[f'z_rolling_std_20'] = df_adj.groupby('symbol')[f'rolling_std_20'].transform(
        lambda x: (x - x.rolling(z_window).mean()) / x.rolling(z_window).std()
    )

In [42]:
# Range Ratio (High - Low) / Close, per asset
df_adj['range_ratio'] = (df_adj['high'] - df_adj['low']) / df_adj['close']
# Adding smoothing (e.g., rolling average of 5 days)
df_adj['range_ratio_smooth'] = df_adj.groupby('symbol')['range_ratio'].transform(lambda x: x.rolling(5).mean())
# Standardizing with Z-scores for regime shift detection
df_adj['z_range_ratio_smooth'] = df_adj.groupby('symbol')['range_ratio_smooth'].transform(lambda x: (x - x.rolling(z_window).mean()) / x.rolling(z_window).std())

df_adj.drop(columns=['range_ratio', 'range_ratio_smooth'], inplace=True)

In [43]:
# Volatility Shock Count
# The number of days in a recent lookback window where daily returns exceeded 2 standard deviations, either up or down.
# It captures the frequency of abnormal moves — a key indicator for risk, panic clusters, or momentum bursts.

# 1. Calculate return z-scores per symbol
df_adj['z_log_returns'] = df_adj.groupby('symbol')['log_returns'].transform(
    lambda x: (x - x.rolling(60).mean()) / x.rolling(60).std()
)

# 2. Count shocks in rolling window (e.g., abs(z) > 2 over last 20 days)
df_adj['vol_shock_count_20'] = df_adj.groupby('symbol')['z_log_returns'].transform(
    lambda x: x.rolling(20).apply(lambda r: (abs(r) > 2).sum(), raw=True)
)

df_adj.drop(columns=['log_returns'], inplace=True)

In [44]:
df_adj = df_adj.sort_values(by=['symbol', 'timestamp'])

In [45]:
df_pivot = df_adj.pivot(columns="symbol")
# Flatten columns ('close', 'QQQ') -> 'close_QQQ'
df_pivot.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_pivot.columns]

In [46]:
# Volatility Cross-Asset Dispersion lets you measure how much the volatility levels themselves are diverging across assets. It's like saying: are
#  all sectors jittery together or are some calm while others are chaotic? Great for reading the market’s internal stress or divergence.

# Step 1: Filter only the 'rolling_std_20' columns
rolling_std_cols = [col for col in df_pivot.columns if col.startswith('rolling_std_20_')]

# Step 2: Compute cross-sectional std (dispersion) across symbols for each timestamp
vol_dispersion = df_pivot[rolling_std_cols].std(axis=1)

# Step 3: Add back into df_pivot as a new column
df_pivot['volatility_dispersion'] = vol_dispersion

In [47]:
# Realized Vol / Implied Vol compares actual historical price movement (realized vol) with what the market expects (implied vol). 
# A ratio >1 means realized volatility has exceeded market expectations — a sign of surprise, dislocation, or a catch-up in pricing. 
# A ratio <1 suggests implied vol is elevated — often in times of uncertainty or hedging demand.

# Z-realized vs Implied Volatility Ratios

# Reference realized vol
realized = df_pivot['rolling_std_20_IWM']

# Loop over each implied asset and compute z-scored realized/implied ratios
implied = df_pivot['rolling_std_20_VIXY']
ratio = realized / implied
z_col = f'z_realized_implied_vixy'

df_pivot[z_col] = (ratio - ratio.rolling(z_window).mean()) / ratio.rolling(z_window).std()


In [48]:
# Volatility Spread = VIXY implied vol proxy – realized vol of SPY/IWM
# Z-score Level	What It Means
# > +2	Implied vol (VIXY) is much higher than realized vol → Fear spike
# < -2	Realized vol is unusually high vs what’s priced in → Complacency mispricing?
# Around 0	Implied and realized vol are in sync → Stable regime

# Step 1: Compute average realized vol of SPY and IWM
realized_avg = df_pivot[['rolling_std_20_SPY', 'rolling_std_20_IWM']].mean(axis=1)

# Step 2–4: Loop over implied assets to compute z-scored spread and clean up
spread = df_pivot['rolling_std_20_VIXY'] - realized_avg
z_col = f'z_vol_spread_vixy'

df_pivot[z_col] = (spread - spread.rolling(z_window).mean()) / spread.rolling(z_window).std()

In [49]:
# Volatility Skew = Difference in realized vol between two symbols
# QQQ vs SPY — Big Tech vs Broad Market - IWM vs SPY — Small Caps vs Broad Market - GLD vs SPY — Gold vs Broad Market
# Define the assets to compare against SPY
skew_assets = ['IWM', 'GLD']

# Loop to compute z-scored volatility skew vs SPY
for ticker in skew_assets:
    spread = df_pivot[f'rolling_std_20_{ticker}'] - df_pivot['rolling_std_20_SPY']
    z_col = f'z_vol_skew_{ticker.lower()}'
    
    df_pivot[z_col] = (spread - spread.rolling(z_window).mean()) / spread.rolling(z_window).std()

In [50]:
# Expansion vs. Contraction
# Advance-Decline Ratio (ADR)

# Step 1: Identify close columns
close_cols = [col for col in df_pivot.columns if col.startswith('close_')]

# Step 2: Calculate previous close using shift
prev_close = df_pivot[close_cols].shift(1)

# Step 3: Calculate change
change = df_pivot[close_cols] - prev_close

# Step 4: Classify movement
advance = (change > 0).astype(int)
decline = (change < 0).astype(int)

# Step 5: Count advances and declines per timestamp
advancing_count = advance.sum(axis=1)
declining_count = decline.sum(axis=1)

# Step 6: Calculate ADR  manual encoding to avoid division by zero

adr_series = pd.Series(index=df_pivot.index, dtype=float)

for i in df_pivot.index:
    adv = advancing_count[i]
    dec = declining_count[i]
    
    if adv == 0 and dec == 0: # avoifd division by zero
        adr_series[i] = 1.0  # neutral market
    elif dec == 0:
        adr_series[i] = len(set(custom_vol_subset + custom_exp_subset))  # strong bullish signal
    else:
        adr_series[i] = adv / dec

# analyzing trends, a rolling average helps
adr_smoothed = pd.Series(adr_series).rolling(window=5).mean()

# Step 7: Add ADR to df_pivot
df_pivot['ADR_smooth'] = adr_smoothed


In [51]:
# Expansion vs. Contraction
# Zweig Breadth Thrust (ZBT) is a classic and powerful breadth thrust indicator used to identify the start of strong bullish moves

# Step 1: Use previously computed advance matrix
# (from your ADR calculation)
# advance = (change > 0).astype(int)

# Step 2: Compute percentage of advancing stocks per timestamp
advancing_percent = advance.sum(axis=1) / advance.shape[1]

# Step 3: Compute 10-day moving average of advancing percent, ema more smooth than sma
zbt_series = advancing_percent.ewm(span=10, adjust=False).mean()
df_pivot['zbt'] = zbt_series

In [52]:
# Expansion vs. Contraction
# % tickers Above MA50/MA200 Market-wide participation
# Step 1: Identify close columns
close_cols = [col for col in df_pivot.columns if col.startswith('close_')]

# Step 2: Compute MA50 and MA200 for each symbol
ma50 = df_pivot[close_cols].rolling(window=50, min_periods=1).mean()
ma200 = df_pivot[close_cols].rolling(window=200, min_periods=1).mean()

# Step 3: Compare close to MA50 and MA200
above_ma50 = (df_pivot[close_cols] > ma50).astype(int)
above_ma200 = (df_pivot[close_cols] > ma200).astype(int)

# Step 4: Compute % of stocks above each MA
pct_above_ma50 = above_ma50.sum(axis=1) / len(close_cols) * 100
pct_above_ma200 = above_ma200.sum(axis=1) / len(close_cols) * 100

# Step 5: Add to df_pivot
df_pivot['Pct_Above_MA50'] = pct_above_ma50
df_pivot['Pct_Above_MA200'] = pct_above_ma200

In [53]:
# Expansion vs. Contraction
# Number of stocks making a new 20-day high, Number of stocks making a new 20-day low, New Highs vs. New Lows Ratio
# Step 1: Identify close columns
close_cols = [col for col in df_pivot.columns if col.startswith('close_')]

# Step 2: Compute rolling 20-day high and low
rolling_high = df_pivot[close_cols].rolling(window=20, min_periods=1).max()
rolling_low = df_pivot[close_cols].rolling(window=20, min_periods=1).min()

# Step 3: Identify new highs and new lows
new_highs = (df_pivot[close_cols] >= rolling_high).astype(int)
new_lows = (df_pivot[close_cols] <= rolling_low).astype(int)

# Step 4: Count per timestamp
nh_count = new_highs.sum(axis=1)
nl_count = new_lows.sum(axis=1)

# Step 5: Compute NH/NL Ratio with manual encoding to avoid division by zero
nh_nl_ratio = pd.Series(index=df_pivot.index, dtype=float)

for i in df_pivot.index:
    nh = nh_count[i]
    nl = nl_count[i]
    
    if nl == 0 and nh == 0: # avoifd division by zero
        nh_nl_ratio[i] = 1.0  # flat market
    elif nl == 0: # if there are no new lows, we have a strong bullish signal, avoid division by zero
        nh_nl_ratio[i] = len(set(custom_vol_subset + custom_exp_subset))  # strong bullish signal
    else:
        nh_nl_ratio[i] = nh / nl # normal ratio


nh_nl_ratio_smoothed = pd.Series(nh_nl_ratio).rolling(window=5).mean()

# Step 6: Add to df_pivot
df_pivot['NH_NL_Ratio_smooth'] = nh_nl_ratio_smoothed

In [54]:
# Expansion vs. Contraction
# Volume Surge feature, which compares total volume at each timestamp to its rolling average
# Step 1: Identify volume columns
volume_cols = [col for col in df_pivot.columns if col.startswith('volume_')]

# Step 2: Compute total volume per timestamp
total_volume = df_pivot[volume_cols].sum(axis=1)

# Step 3: Compute rolling average volume (e.g., 20-day)
rolling_avg_volume = total_volume.rolling(window=20, min_periods=1).mean()

# Step 4: Compute volume surge ratio
volume_surge_ratio = total_volume / rolling_avg_volume

# Step 5: Manual encoding to avoid NaN or inf
volume_surge = pd.Series(index=df_pivot.index, dtype=float)

for i in df_pivot.index:
    tv = total_volume[i]
    rv = rolling_avg_volume[i]
    
    if rv == 0:
        volume_surge[i] = 1.0  # neutral if no prior volume
    else:
        volume_surge[i] = tv / rv

df_pivot['volum_surge'] = volume_surge

In [55]:
# Drop 'SPY' columns to avoid redundancy, they where just used for comparison
df_pivot.drop(columns=[col for col in df_pivot.columns if 'SPY' in col], inplace=True)

# Drop 'rolling_std_20' columns to avoid redundancy, they where just used for comparison, we keep z-scores only
# Drop columns that start with 'rolling_std_20'
df_pivot.drop(columns=[col for col in df_pivot.columns if col.startswith("rolling_std_20")], inplace=True)

In [56]:
# after 2*max(windows) +z_window -1 days, not nans
print(df_pivot[max(windows)+z_window-1:].isna().sum().sum()) 

df_pivot_clean = df_pivot[max(windows)+z_window-1:]

0


In [57]:
df_stationary = df_pivot_clean.drop(
    columns=[col for col in df_pivot_clean.columns if any(x in col for x in ["vwap", "low", "high", "close", "open", "volume",
                                                                              "trade_count", "z_log_returns", "z_range_ratio_smooth", "z_rolling", "vol_shock"])]
)

In [58]:
df_stationary.columns

Index(['volatility_dispersion', 'z_realized_implied_vixy', 'z_vol_spread_vixy',
       'z_vol_skew_iwm', 'z_vol_skew_gld', 'ADR_smooth', 'zbt',
       'Pct_Above_MA50', 'Pct_Above_MA200', 'NH_NL_Ratio_smooth',
       'volum_surge'],
      dtype='object')

In [59]:
N_COMP=3
N_NEIG=15
MIN_D = 0.01
METRIC = "correlation" 

In [ ]:
#####################
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import umap
import pandas as pd
import numpy as np

# Define column list
columns = [
    'volatility_dispersion', 'z_realized_implied_vixy', 'z_vol_spread_vixy',
    'z_vol_skew_iwm', 'z_vol_skew_gld', 'ADR_smooth', 'zbt',
    'Pct_Above_MA50', 'Pct_Above_MA200', 'NH_NL_Ratio_smooth',
    'volum_surge'
]

# Generate all subsets of length 8 to 11
column_subsets = [list(subset) for r in range(8, len(columns)+1) for subset in combinations(columns, r)]

# Store results
results = []
counter = 0

for subset in column_subsets:
    df_subset = df_stationary[subset]
    scaled = MinMaxScaler().fit_transform(df_subset)
    X_umap = umap.UMAP(n_neighbors=N_NEIG, min_dist=MIN_D, n_components=N_COMP, metric=METRIC).fit_transform(scaled)

    for k in [3, 4]:
        gmm = GaussianMixture(n_components=k, random_state=0)
        gmm_labels = gmm.fit_predict(X_umap)

        result = {
            "subset": subset,
            "k": k,
            "silhouette": silhouette_score(X_umap, gmm_labels),
            "calinski_harabasz": calinski_harabasz_score(X_umap, gmm_labels),
            "davies_bouldin": davies_bouldin_score(X_umap, gmm_labels)
        }
        results.append(result)
        print(counter, len(column_subsets))
        counter+=1

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df_sorted = results_df.sort_values(by="silhouette", ascending=False)
results_df_sorted.to_csv("gmm_subset_cols.csv", index=False)
###################

0 232
1 232
2 232
3 232
4 232
5 232
6 232
7 232
8 232
9 232
10 232
11 232
12 232
13 232
14 232
15 232
16 232
17 232
18 232
19 232
20 232
21 232
22 232
23 232
24 232
25 232
26 232
27 232
28 232
29 232
30 232
31 232
32 232
33 232
34 232
35 232
36 232
37 232
38 232
39 232
40 232
41 232
42 232
43 232
44 232
45 232
46 232
47 232
48 232
49 232
50 232
51 232
52 232
53 232
54 232
55 232
56 232
57 232
58 232
59 232
60 232
61 232
62 232
63 232
64 232
65 232
66 232
67 232
68 232
69 232
70 232
71 232
72 232
73 232
74 232
75 232
76 232
77 232
78 232
79 232
80 232
81 232
82 232
83 232
84 232
85 232
86 232
87 232
88 232
89 232
90 232
91 232
92 232
93 232
94 232
95 232
96 232
97 232
98 232
99 232
100 232
101 232
102 232
103 232
104 232
105 232
106 232
107 232
108 232
109 232
110 232
111 232
112 232
113 232
114 232
115 232
116 232
117 232
118 232
119 232
120 232
121 232
122 232
123 232
124 232
125 232
126 232
127 232
128 232
129 232
130 232
131 232
132 232
133 232
134 232
135 232
136 232
137 232
138 23

In [ ]:
# Optional: sort by silhouette score
results_df_sorted = results_df.sort_values(by="silhouette", ascending=False)
results_df_sorted.head(20)

In [ ]:
from sklearn.cluster import KMeans
import umap
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    silhouette_score, calinski_harabasz_score, davies_bouldin_score
)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



# Range of K values to test
k = 3

# Scale and apply UMAP
scaled = MinMaxScaler().fit_transform(df_stationary)
X_umap = umap.UMAP(n_neighbors=N_NEIG, min_dist=MIN_D, n_components=N_COMP, metric=METRIC).fit_transform(scaled)

silhouette_scores = []
ch_scores = []
db_scores = []

gmm = GaussianMixture(n_components=k, random_state=0)
gmm_labels = gmm.fit_predict(X_umap)

print("gmm", {
    "silhouette": silhouette_score(X_umap, gmm_labels),
    "calinski_harabasz": calinski_harabasz_score(X_umap, gmm_labels),
    "davies_bouldin": davies_bouldin_score(X_umap, gmm_labels)
})


In [ ]:

# Define component pairs for plotting
component_pairs = [(0, 1), (0, 2), (1, 2)]
titles = [
    "UMAP Component 1 vs 2",
    "UMAP Component 1 vs 3",
    "UMAP Component 2 vs 3"
]

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

def plot_gmm_std_ellipses(ax, gmm, comp_x, comp_y, colors):
    from scipy.stats import chi2

    std_levels = [1, 2, 3]
    chi2_vals = [chi2.ppf(0.3935, df=2), chi2.ppf(0.8647, df=2), chi2.ppf(0.9889, df=2)]
    scales = [np.sqrt(val) for val in chi2_vals]

    for i, (mean, covar) in enumerate(zip(gmm.means_, gmm.covariances_)):
        if covar.ndim == 1:  # spherical
            cov = np.diag(covar)
        elif covar.ndim == 2:  # full or tied
            cov = covar
        else:
            continue

        # Project to 2D
        sub_cov = cov[np.ix_([comp_x, comp_y], [comp_x, comp_y])]
        sub_mean = mean[[comp_x, comp_y]]

        # Eigen decomposition
        vals, vecs = np.linalg.eigh(sub_cov)
        order = vals.argsort()[::-1]
        vals, vecs = vals[order], vecs[:, order]
        theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

        for scale, std in zip(scales, std_levels):
            width, height = 2 * scale * np.sqrt(vals)

            if std == 1:
                linestyle = '-'
            elif std == 2:
                linestyle = '--'
            else:
                linestyle = ':'

            ellipse = Ellipse(
                sub_mean, width, height, angle=theta,
                edgecolor=colors[i], facecolor='none', lw=1.5, linestyle=linestyle
            )
            ax.add_patch(ellipse)


# Define colors for ellipses
colors = plt.cm.tab10(np.linspace(0, 1, k))

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (comp_x, comp_y) in enumerate(component_pairs):
    ax = axes[i]
    scatter = ax.scatter(
        X_umap[:, comp_x], X_umap[:, comp_y],
        c=gmm_labels, cmap='tab10', s=40, alpha=0.7
    )
    ax.set_xlabel(f'UMAP Component {comp_x + 1}')
    ax.set_ylabel(f'UMAP Component {comp_y + 1}')
    ax.set_title(titles[i])
    ax.grid(True)

    # Plot GMM ellipses
    plot_gmm_std_ellipses(ax, gmm, comp_x, comp_y, colors)

# Add legend to the last plot
from matplotlib.lines import Line2D
custom_lines = [
    Line2D([0], [0], color='black', lw=1.5, linestyle='-'),
    Line2D([0], [0], color='black', lw=1.5, linestyle='--'),
    Line2D([0], [0], color='black', lw=1.5, linestyle=':')
]
axes[-1].legend(custom_lines, ['1σ (~39%)', '2σ (~86%)', '3σ (~99%)'], title="Confidence Ellipses")


plt.tight_layout()
plt.show()


In [ ]:
### creo que es mejor entrenar un modelo para cada combinacion de clusters. kmeans con Wasserstein distance nos da hard labels, mas simple,
# GMM nos da soft labels (probs), mas complejo, pero podria apañar algo como threholds the probs, es decir un modelo para cuando la probabilidad de 
# cluster 1 es mayor que 0.5, otro para cuando es mayor que 0.7, etc.

In [ ]:
print("\n--- Explained Variance Ratio ---")
print(pca.explained_variance_ratio_)
print(f"Total variance explained by {X_pca.shape[1]} components: {pca.explained_variance_ratio_.sum():.2f}")

In [ ]:
## BUSCAR UN GRUPO QUE NO TENGA UNA CORRELACION ALTA?????

# # # ##### Correlation Heatmap #####
import seaborn as sns
import matplotlib.pyplot as plt


# Optionally drop rows with missing values to ensure clean correlation
corr_df = df_stationary.dropna()

# Compute the correlation matrix
correlation_matrix = corr_df.corr()

# Plot the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Matrix of Close Prices by Symbol")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))

for symbol in ['UVXY', 'VIXY']:
    subset = df_adj[df_adj['symbol'] == symbol]
    ax.plot(subset.index, subset['log_returns'], label=symbol)

ax.set_title('Close Prices: PDBC vs DBC')
ax.set_xlabel('Date')
ax.set_ylabel('Close Price')
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()
